<h1> Wykład 6 - Geometrie, obiekty, układy współrzędnych </h1>




Dotychczas mówiliśmy o klasach:


1. QgsProject - klasa projektu

2. QgsVectorLayer - klasa warstwy wektorowej

3. QgsRaterLayer - klasa warstwy rastrowej

4. QgsFeature - klasa obiektu warstwy wektorowej

5. QgsField - klasa pola wartości w atrybutach warstwy wektorowej

6. QgsGeometry - klasa geometrii obiektu

7. iface - odwołanie do klasy interfejsu aplikacji,

Klasy geometrii:

8. QgsPoint, QgsPointXY,

9. QgsPolygonXY

10. QgsPolylineXY

Istotne metody:

    .dataProvider() - wywołanie klasy dostępu do atrybutów i geometrii warstwy

Sposoby tworzenia geometrii:

    z koordynatów:
        
        punkt = QgsGeometry.fromPointXY(QgsPointXY(1,1))
        
        linia = QgsGeometry.fromPolyline([QgsPoint(1,1), QgsPoint(2,2)])
        
        polygon = QgsGeometry.fromPolygonXY([[QgsPointXY(1,1),QgsPointXY(2,2), QgsPointXY(2,1)]])

Różnica między QgsPoint a QgsPointXY jest taka, że pierwsza wspiera wymiar Z lub M

Z WKT:

gem = QgsGeometry.fromWkt("POINT(3 4)")

Z WKB:

wkb = bytes.fromhex("010100000000000000000045400000000000001440")
g.fromWkb(wkb)

Geometrię jako WKT:

geometria.asWkt()

<h3> Dostęp do geometrii: </h3>

    geometria.asPoint()
    
    geometria.asPolyline()
    
    geometria.asPolygon()
    

Porównanie geometri z typami:

Wykorzystujemy do tego zapytanie wkbType(), które należy porównać z enumeracją w QgsWkbTypes:

    gPnt.wkbType() == QgsWkbTypes.Point
    
    gLine.wkbType() == QgsWkbTypes.LineString
    
    gPolygon.wkbType() == QgsWkbTypes.Polygon
    
    gPolygon.wkbType() == QgsWkbTypes.MultiPolygon

<h3> Operatory geometrii </h3>

QGIS umożliwa dostęp do operatorów geometrycznych wprost jako metody. Najwazniejsze:

    area - zwraca powierzchnię
    length - zwraca długość obiektu
    centroid - zwraca centroid obiektu
    
    buffer - tworzy bufor
    combine - zwraca sumę obiektów
    intersection - zwraca przeciecie obiektow
    
    compare - porównuje geometrię
    contains - sprawdza czy jeden obiekt zawiera drugi
    crosses - sprawdza czy jeden obiekt przecina drugi
    equal - sprawdza czy obiekty sa takie same
    disjoint - sprawdza czy obiekty sa rozlaczne
    distance - sprawdza czy obiekty sa w zakresie pewnego dystanu
    intersects - sprawdza czy obiekty sie przecinaja
    overlaps - sprawdza czy obiekty sie nakladaja
    touches - sprawdza czy obiekty sie stykają
    within - sprawdza czy obiekt jest zawarty w innym obiekcie

In [ ]:
warstwa = iface.activeLayer()
features = [obiekt for obiekt in warstwa.getFeatures()]
for x in features:
    print(x.geometry().area())
    

In [ ]:
warstwa = iface.activeLayer()
features = [obiekt for obiekt in warstwa.getFeatures()]
for x in features:
    print(x.geometry().length())
    

In [ ]:
#Z dodaniem warstwy tymczasowej

warstwa = iface.activeLayer()
features = [obiekt.geometry() for obiekt in warstwa.getFeatures()][:5]
nowe = []
for x in features:
    new_geom = x.buffer(0.0004,-1)
    nowe.append(new_geom)
    
mem_layer = QgsVectorLayer("Polygon","Buffer","memory")
pr = mem_layer.dataProvider()
for n in range(len(nowe)):
    bufor = QgsFeature(n)
    bufor.setGeometry(nowe[n])
    pr.addFeatures([bufor])
QgsProject.instance().addMapLayer(mem_layer)

Problem z jednostkami kątowymi - jak przejść na metry?

Można użyć od razu warstwy w układzie metrycznym. Co w sytuacji, kiedy jednak jesteśmy w układzie WGS84?

1. Konwertować wszystkie warstwy do układu metrycznego poprzez ich zapis,
2. Konwertować tymczasowo w projekcie,
3. W przypadku długości, odległości skorzystać z klasy QgsDistanceArea

In [ ]:
warstwa = iface.activeLayer()
features = [obiekt.geometry() for obiekt in warstwa.getFeatures()]

d = QgsDistanceArea()
d.setEllipsoid('WGS84')

for x in features:
    print(d.measureArea(x)) #domyślnie w metrach

In [ ]:
warstwa = iface.activeLayer()
features = [obiekt.geometry() for obiekt in warstwa.getFeatures()][:5]

d = QgsDistanceArea()
d.setEllipsoid('WGS84')
d.areaUnits()

for x in features:
    pomiar = d.measureArea(x)
    akry = d.convertAreaMeasurement(pomiar, QgsUnitTypes.AreaUnit(6)) #w akrach
    print(akry)

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "1.png")

<h3> Transformacje </h3>

Klasa (!)
     
    QgsCoordinateReferenceSystem

In [ ]:
uklad = QgsCoordinateReferenceSystem(4326, QgsCoordinateReferenceSysten.PostgisCrsId)
#Tworzy uklad na podstawie id jak w PostGIS

In [ ]:
wgs = QgsCoordinateReferenceSystem(4326,QgsCoordinateReferenceSystem.EpsgCrsId)
#Na podstawie EPSG


Więcej informacji:

    uklad.srsid()
    
    uklad.postgisSrid()
    
    uklad.description()
    
    uklad.mapUnits()

In [ ]:
z = QgsCoordinateReferenceSystem(4326) #domyślnie EPSG
do = QgsCoordinateReferenceSystem(2177) #uklad2000

transformacja = QgsCoordinateTransform(z,do, QgsProject.instance()) 
#przygotowujemy transformację w projekcie

warstwa = iface.activeLayer()
features = [obiekt.geometry() for obiekt in warstwa.getFeatures()][:5]
for x in features:
    nowy = transformacja.transform(x.asPoint())
    print(nowy)


In [ ]:
z = QgsCoordinateReferenceSystem(4326) #domyślnie EPSG
do = QgsCoordinateReferenceSystem(2177) #uklad2000

transformacja = QgsCoordinateTransform(z,do, QgsProject.instance()) 
#przygotowujemy transformację w projekcie

warstwa = iface.activeLayer()
nowa_warstwa = QgsVectorLayer("Polygon","nowa","memory")
nowa_warstwa.setCrs(do) #a co będzie bez?
features = [obiekt for obiekt in warstwa.getFeatures()][:5]
for x in features:
    geometria = x.geometry()
    geometria.transform(transformacja)
    x.setGeometry(geometria)
    nowa_warstwa.dataProvider().addFeatures([x])
QgsProject.instance().addMapLayer(nowa_warstwa)